In [3]:
import numpy as np
import pandas as pd
import networkx as nx
#from stellargraph import StellarGraph
#from stellargraph.data import BiasedRandomWalk
#from gensim.models import Word2Vec
import csv

In [5]:
#Get train.csv data
df = pd.read_fwf('train.csv', header=None)
df = df[0].str.split(',', expand=True) 

In [6]:
#Get ingredient id
df2 = pd.read_fwf('node_ingredient.csv', header=None)
idToNode = {}
for index , row in df2.iterrows():
    idToNode[index] = row[0]


In [8]:
idToNode

{0: 'coca-cola',
 1: 'vegan butter',
 2: 'sourdough rolls',
 3: 'reduced sodium refried beans',
 4: 'ramen noodles',
 5: 'crumbled corn bread',
 6: 'japanese breadcrumbs',
 7: 'toasted shredded coconut',
 8: 'chinese spinach',
 9: "Hellmann's® Real Mayonnaise",
 10: 'ducklings',
 11: 'basil olive oil',
 12: 'white baking bar',
 13: 'rye whiskey',
 14: 'mushroom broth',
 15: 'meat loaf mix',
 16: 'cocktail sauce',
 17: 'asparagus spears',
 18: 'nonfat greek yogurt',
 19: 'cabernet sauvignon',
 20: 'low sodium tomato sauce',
 21: 'shredded colby',
 22: 'dried dillweed',
 23: 'frozen potatoes',
 24: 'hot dog bun',
 25: 'balm leaves',
 26: 'lo mein noodles',
 27: 'green split peas',
 28: 'celery leaves',
 29: 'butterflied leg of lamb',
 30: 'refrigerated piecrusts',
 31: 'cajun spice mix',
 32: 'finely chopped onion',
 33: 'chili bean sauce',
 34: 'brown rice',
 35: 'cutlet',
 36: 'beef jerky',
 37: 'lemon grass',
 38: 'ground walnuts',
 39: 'kinchay',
 40: 'macarons',
 41: 'crème de cassi

In [7]:
len(idToNode)

6714

In [ ]:
#Create dict of dict where the first key is ingredient id the value is another dict 
#with key are nodes he shares receipt with and value the number of receipt they have in common
ingredientAdjancy = {}
ingredientNbR = {}
for index , row in df.iterrows():
    for i in row.values:
        if i is not None and i.isdigit() and int(i) not in ingredientAdjancy.keys():
            ingredientAdjancy[int(i)] = {}
        for y in row.values:
            if i is not None and  y is not None and i.isdigit() and y.isdigit():
                if i != y:
                    if int(y) not in ingredientAdjancy[int(i)].keys():
                        ingredientAdjancy[int(i)][int(y)] = 1
                    else:
                        ingredientAdjancy[int(i)][int(y)] += 1
        if i is not None and i.isdigit():
            if int(i) not in ingredientNbR.keys():
                ingredientNbR[int(i)] = 1
            else:
                ingredientNbR[int(i)] += 1


In [ ]:
ingredientAdjancy

In [ ]:
with open('edgeListWeight.csv','w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    for i in ingredientAdjancy.keys():
        for y in ingredientAdjancy[i].keys():
            writer.writerow([i, y, ingredientAdjancy[i][y]])

In [ ]:
with open('edgeList.csv','w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    for i in ingredientAdjancy.keys():
        for y in ingredientAdjancy[i].keys():
            writer.writerow([i, y])

In [ ]:
Gw = nx.read_edgelist('edgeListWeight.csv', delimiter=',', nodetype=int, data=(('weight',float),))
G = nx.read_edgelist('edgeList.csv', delimiter=',', nodetype=int, data=(('weight',float),))

In [ ]:
print(len(G.nodes()))
print(len((G.edges())))
nodes = list(G.nodes())


In [ ]:
import os
os.getcwd()

In [ ]:
pd.read_csv('Embedding/Embp1q10.csv', delimiter=",")

Some ingredient are not part of the graph (have no edge)

Use Node2vec -> first need to generate weighted random walk. Then use Word2Vec to generate the embedding.
Generate random walktake quite some time, I already generate 4 of them using different paramters that should adjust the balance between sutrucural and local importance. This might be improved by trying different set of hyperparameter. 

In [ ]:
#Use StellarGraph library to generate the weighted random walk
GwS = StellarGraph.from_networkx(Gw)

In [ ]:
rw = BiasedRandomWalk(GwS)

In [ ]:
weighted_walks = rw.run(
        nodes=G.nodes(), 
        length=100,    # maximum length of a random walk
        n=10,          # number of random walks per root node
        p=1,         # Defines (unormalised) probability, 1/p, of returning to source node
        q=100,         # Defines (unormalised) probability, 1/q, for moving away from source node
        weighted=True, #for weighted random walks
        seed=42        # random seed fixed for reproducibility
    )

In [ ]:
str(weighted_walks[1])

In [ ]:
#Save the random walk
with open('NodeRandomWalkp1q100.csv','w', newline='') as fh:
    writer = csv.writer(fh, delimiter=',')
    for i in range(len(G.nodes())):
        for y in range(10):
            writer.writerow([str(GwS.nodes()[i]), weighted_walks[i*10+y]])

In [ ]:
#Import random walk
import ast
rp1q10 = pd.read_fwf('RW/WRandomWalkp1q2.csv', header=None, delimiter= " ")

rp1q10_list = []
for index, value in rp1q10.iterrows():
    t = False
    s = ""
    for i in value[0]:
        if i == "[":
            t = True
        if i == '"' and t:
            t = False
        if t:
            s+=i
    if s[len(s)-1] != "]":
        s+="]"
    rp1q10_list.append(ast.literal_eval(s)) 
  

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

embNod2V = Word2Vec(sentences=rp1q10_list, vector_size=64, window=3, min_count=0, sg=1, workers=1, epochs=1)

In [ ]:
len(embNod2V.wv)

In [ ]:
from numpy import dot
from numpy.linalg import norm
m=0
test = 6000
for i in Gw.nodes():
    if i != test:
        a = embNod2V.wv[test]
        b = embNod2V.wv[i]
        cos_sim = dot(a, b)/(norm(a)*norm(b))
        if cos_sim > m:
            vm = i
            m = cos_sim
print(m,vm)
print(idToNode[test]," is most similar to: ",idToNode[vm])

In [ ]:
fh = open('Embedding/Embp1q100.csv','w')
writer = csv.writer(fh)
for i in Gw.nodes():
    writer.writerow([i,embNod2V.wv[i]])
fh.close()

Second embedding method (not weighted)

In [ ]:
from sklearn.decomposition import  TruncatedSVD

A = nx.adjacency_matrix(G) 

svd = TruncatedSVD(n_components=128)
X = svd.fit_transform(A);

Save different embedding size (You can find 32-64-128 in /embedding)

In [ ]:
import csv

fh = open('Embedding/SVD128.csv','w')
writer = csv.writer(fh)

y = 0
for i in G.nodes():
    writer.writerow([i,X[y]]) 
    y += 1
    
fh.close()

In [ ]:
test = 6
m = 0

for i in range(5860):
    if i != test:
        a = X[test]
        b = X[i]
        cos_sim = dot(a, b)/(norm(a)*norm(b))
        if cos_sim > m:
            vm = i
            m = cos_sim
print(m,vm)
print(idToNode[list(G.nodes())[test]]," is most similar to: ",idToNode[list(G.nodes())[vm]])